<a href="https://colab.research.google.com/github/LoPA607/AIDS_248/blob/main/hello_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello Bayes!

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

## Recall some theory

- Let $H$ be the hypothesis, let $E$ be the evidence or data, then recall that:

$$
f(H|E) = \frac{f(E|H) f(H)}{f(E)}   
$$

- Specifically for an unknown parameter $\theta$ and
- A sample with data $x_1,\ldots,x_n$,
- We have

$$\begin{aligned}
f(\theta|x_1,\ldots,x_n) &=
\frac{f(\theta, x_1, \ldots, x_n)}{f(x_1, \ldots, x_n)} \\
&= \frac{p(\theta)f(x_1, \ldots, x_n|\theta)}{\int f(x_1, \ldots,
x_n|\theta) p(\theta) d\theta}
\end{aligned}$$

## Problem

* Consider a coin which may be biased or not.
* We want to assess the parameter, $p$ but using a Bayesian approach.
* $p$ is our unknown $\theta$
* Want to estimate the distribution of $p$ from an initial guess/belief and data

## Generating the data

In [ ]:
N = 50
data = np.random.choice([0, 1], size=N)

## Setting up the problem

* Assume a prior, we start with a uniform distribution
* We need the likelihood function: $f(x| \theta) = \theta^x (1-\theta)^{1-x}$
* Samples are independent so we can easily construct the joint likelihood.
* We solve the problem by numerical integration
* Recall that we are looking to find $\theta$
    * Fix the domain to 100 points between 0 and 1.

## Back to the problem at hand

In [ ]:
nt = 100
d_theta = 1.0/nt
domain = np.arange(d_theta/2, 1, d_theta)

In [ ]:
def likelihood(theta, data):
    """Assumes that theta is a scalar"""
    return np.prod(np.pow(theta, data)*np.pow(1-theta, 1-data))

@np.vectorize
def prior_func(theta):
    if (theta < 0.0) or (theta > 1.0):
        return 0.0
    else:
        return 1.0

In [ ]:
def integral_dnr(l, prior, dtheta):
    """Assumes that l and prior are uniformly discretized"""
    return np.sum(l*prior)*dtheta


In [ ]:
lh = []
for t in domain:
    lh.append(likelihood(t, data))
lh = np.array(lh)

## Aside: list comprehensions

In [ ]:
[i*i for i in range(5)]

In [ ]:
[(i, i*i) for i in range(5)]

In [ ]:
[i*i for i in range(5) if i%2 == 0]

## Back to the problem

In [ ]:
lh = np.array([likelihood(t, data) for t in domain])

In [ ]:
prior = prior_func(domain)

In [ ]:
dnr = integral_dnr(lh, prior, d_theta)

In [ ]:
posterior = prior*lh/dnr

In [ ]:
plt.plot(domain, posterior)

## Textbook answer

- $x=\sum_i x_i$

$$f(\theta|x_1,\ldots,x_n) = \frac{(n+1)! \theta^x (1-\theta)^{n-x}}{x! (n-x)!}$$


In [ ]:
from scipy.special import factorial

def f_exact(theta, x, n):
    return factorial(n+1)*theta**x*(1-theta)**(n-x)/(factorial(x)*factorial(n-x))

In [ ]:
plt.plot(domain, posterior, label='Computed')
x = np.sum(data)
plt.plot(domain, f_exact(domain, x, N), label='Exact')
plt.legend();

## Bayes estimator

- Best estimate of $\theta$ is the mean of the posterior:

$$E[\theta|X_1=x_1, \ldots, X_n=x_n] = \int \theta f(\theta|x_1,\ldots,x_n)
d\theta$$

In [ ]:
# Homework

## Textbook answer

$$E[\theta|X_1=x_1, \ldots, X_n=x_n] = \frac{x+1}{n+2}$$

## Aside: doing this a bit faster

- Looping to get the likelihoods is OK but not so nice
- It can also be slow
- We can instead use numpy broadcasting smartly to do this better.

## Broadcasting rules

- Consider `A <operator> B`

- Compare their shapes element-wise starting with the rightmost dimension
  going left.

- Two dimensions are compatible if:

  - they are equal
  - or one of them is 1

- Do not need the same number of dimensions

## Examples

- From the documentation!

```
Image  (3d array): 256 x 256 x 3
Scale  (1d array):             3
Result (3d array): 256 x 256 x 3
```

```
A      (4d array):  8 x 1 x 6 x 1
B      (3d array):      7 x 1 x 5
Result (4d array):  8 x 7 x 6 x 5
```

## More examples

In [ ]:
a = np.fromfunction(lambda i, j: 10*i, (4, 3))
a

In [ ]:
b = np.array([1.0, 2.0, 3.0, 4.0])
a + b

In [ ]:
b = np.array([1.0, 2.0, 3.0, 4.0]).reshape((4, 1))
a + b

## Introducing a new axis

- Use `np.newaxis` or `None`

In [ ]:
a = np.array([0.0, 10.0, 20.0, 30.0])
b = np.array([1.0, 2.0, 3.0])

In [ ]:
a[:, np.newaxis] + b

In [ ]:
a[:, None] + b

## Back to our problem

Instead of looping to get the likelihood, we can use broadcasting

In [ ]:
data = np.random.choice([0, 1], size=5)
theta = 0.1
np.prod(np.pow(theta, data)*np.pow(1-theta, 1-data))

In [ ]:
theta = np.array([0.1, 0.2])
btheta = theta[:,None]
print(btheta.shape)
print(btheta)

In [ ]:
(btheta*data).shape

In [ ]:
np.prod(np.pow(btheta, data)*np.pow(1-btheta, 1-data), axis=1)

- Can understand this better by looking at:

In [ ]:
np.broadcast_arrays(btheta, data)

## Using this for calculation

In [ ]:
def likelihood(theta, data):
    btheta = theta[:,None]
    return np.prod(np.pow(btheta, data)*np.pow(1-btheta, 1-data), axis=1)

In [ ]:
def calc_posterior(theta, data, prior, likelihood):
    """Assumes that theta is uniformly spaced."""
    lh = likelihood(theta, data)
    d_theta = theta[1] - theta[0]
    dnr = integral_dnr(lh, prior, d_theta)
    post = lh*prior/dnr
    return post

In [ ]:
posterior = calc_posterior(domain, data, prior_func(domain), likelihood)

In [ ]:
plt.plot(domain, posterior)

## Incremental updates of posterior

- Can add data to update the posterior

In [ ]:
data = np.random.choice([0, 1], size=5)

In [ ]:
post = calc_posterior(domain, data, post, likelihood)

In [ ]:
plt.plot(domain, post)